get file paths
get ROIs
concatenate them into a list
KMeans on total
get labels
plot

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 16})
import numpy as np
import matplotlib.pyplot as plt
import csv
from tifffile import imread
import matplotlib.gridspec as gridspec
import cv2
import os
import glob
from sklearn.cluster import KMeans
from matplotlib.colors import LinearSegmentedColormap
import peakutils
from skimage import exposure
# import pandas as pd
from sklearn.cluster import KMeans
import sklearn.preprocessing
import sklearn.decomposition #for PCA

# pca = sklearn.decomposition.PCA(n_components=2)

# perform_PCA = False

In [3]:
#per trial there are 177 rois for this batch
batch_folder_path = 'C:/Users/mmart/Documents/Analysis/FilesToAnalyze/Batch testing/Mar.5.20_fish1_batch/'
main1   = batch_folder_path+'Mar.5.20_HUC6f_8dpf_WFtoBF_1D-1/'
video1  = main1+'Mar.5.20_HUC6f_8dpf_WFtoBF_1D-1_mc.tif'
roi1    = main1+'roi_data.npy'
main2   = batch_folder_path+'Mar.5.20_HUC6f_8dpf_WFtoBF_1C-1/'
video2  = main2+'Mar.5.20_HUC6f_8dpf_WFtoBF_1C-1_mc.tif'
roi2    = main2+'roi_data.npy'
video = imread(video1)
roi_paths = [roi1, roi2]
n_clusters = 4
# main_path  = 'C:/Users/mmart/Documents/Analysis/FilesToAnalyze/Batch testing/Mar.5.20_fish1_batch/Mar.5.20_HUC6f_8dpf_WFtoBF_1D-1/'
# video_name = "Mar.5.20_HUC6f_8dpf_WFtoBF_1D-1_mc.tif"
# video_path = main_path+video_name
# roi_path   = main_path+"roi_data.npy"
# tail_path  = main_path+"Mar.5.20_1D_tail_angles1.csv"
# # Mar.5.20_HUC6f_8dpf_loom_lv30_1B-1_mc.tif
# rois = np.load(roi_path, allow_pickle=True)
# video = imread(video_path)

# the important part - run the function to create the plot
# create_plot(video_path, roi_path, tail_path, video_name, save_dir=main_path) WILL BECOME SOMETHING NEW

In [4]:
def get_batched_data(roi_paths):
    batched_original_data_dict = {}
    trial_names = list(input('Enter trials sequentially by letter: '))
    for i,j in zip(trial_names, roi_paths):
        roi_path = j
        rois        = np.load(roi_path, allow_pickle=True)
#         video       = imread(video_path)

#         prefix = os.path.basename(video_path)[:-4]

        temporal_footprints   = rois[()]['roi_temporal_footprints']
#         temp_residuals        = rois[()]['roi_temporal_residuals']
#         spatial_footprints    = rois[()]['roi_spatial_footprints']
#         bg_temp_footprints    = rois[()]['bg_temporal_footprints']
#         bg_spatial_footprints = rois[()]['bg_spatial_footprints']
        removed_rois          = rois[()]['all_removed_rois']
        
        batched_original_data_dict[i] = [temporal_footprints, removed_rois]
        
    return batched_original_data_dict, trial_names



In [5]:
def get_batched_total_sorted_rois(batched_original_data_dict, video):
# (video, temporal_footprints, removed_rois, n_clusters):
    '''Sorts the ROIS by temporal trace by plane, concatenates them in a list, performs K-Means clustering on all of the kept ROIs
    concatenated list and a list of total sorted ROIs labelled by cluster is returned.'''    
    batch_total_sorted_temporal_traces = []
    batch_total_sorted_kept_rois = []
   
    for trial in batched_original_data_dict: #this is 'd', 'c
        total_trial_sorted_temporal_traces = []
        total_trial_sorted_kept_rois = []
        temporal_footprints = batched_original_data_dict[trial][0]
        temp_footprints = temporal_footprints[:] #prob going to need batched_original_data_dict[trial].temporal_footprints    
    
        for z in range(video.shape[1]):                
            kept_rois = [ i for i in range(batched_original_data_dict[trial][0][z].shape[0]) if i not in batched_original_data_dict[trial][1][z]]

            zscoreNum = temporal_footprints[z][kept_rois] - np.mean(temporal_footprints[z][kept_rois], axis=1)[:, np.newaxis]
            zscorePreDen = np.std(temporal_footprints[z][kept_rois], axis=1)[:, np.newaxis]
            preDenBoolArr = np.isnan(zscorePreDen)
            mmzscoreDen = np.where(zscorePreDen != 0, zscorePreDen, 1)# replaces 0 with 1 so the denominator isn't 0
            zscore = zscoreNum / mmzscoreDen

            # Compute sorted data based on correlations of z-scored data
            correlations = np.corrcoef(zscore)
            i, j = np.unravel_index(correlations.argmin(), correlations.shape)

            temp_footprints[z] = temporal_footprints[z][kept_rois]
            temp_footprints[z][0] = zscore[i]
            total_trial_sorted_temporal_traces.append(zscore[i])
            temp_footprints[z][-1] = zscore[j]

            sorted_kept_rois = kept_rois[:]

            sorted_kept_rois[0] = kept_rois[i]
            sorted_kept_rois[-1] = kept_rois[j]

            remaining_indices = [ index for index in range(temp_footprints[z].shape[0]) if index not in (i, j) ]

            for k in range(1, temp_footprints[z].shape[0]-1):
                corrs_1 = [ correlations[i, index] for index in remaining_indices ]
                corrs_2 = [ correlations[j, index] for index in remaining_indices ]

                difference = [ corrs_1[l] - corrs_2[l] for l in range(len(remaining_indices)) ]
                l = np.argmax(difference)
                index = remaining_indices[l]

                temp_footprints[z][k] = zscore[index]

                sorted_kept_rois[k] = kept_rois[index]

                total_trial_sorted_temporal_traces.append(temp_footprints[z][k])

                del remaining_indices[l]
            total_trial_sorted_temporal_traces.append(zscore[j])
            for roi in sorted_kept_rois:
                total_trial_sorted_kept_rois.append(str(trial)+str(z)+'z'+str(roi))
        
        
        batch_total_sorted_temporal_traces.extend(total_trial_sorted_temporal_traces)
        batch_total_sorted_kept_rois.extend(total_trial_sorted_kept_rois)
        #chose extend() instead of append() because I wanted one full list
        
    return batch_total_sorted_temporal_traces, batch_total_sorted_kept_rois


In [6]:
def elbow_plot():
    pass

In [7]:
def kMeans_clustering(batch_total_sorted_temporal_traces, batch_total_sorted_kept_rois, n_clusters):
#     n_clusters = n_clusters
    kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters) # n=3 is arbitrary, will perform other calculations to figure it out
    clustered_data = kmeans.fit(batch_total_sorted_temporal_traces)

    batch_labelled_total_list = kmeans.labels_
    
    grouped_KMeans_dict = {}
    for label, roi_ident in zip(batch_labelled_total_list, batch_total_sorted_kept_rois):
        if label not in grouped_KMeans_dict:
            roi_list = [roi_ident]
            grouped_KMeans_dict[label] = roi_list
        else:
            roi_list = [roi_ident]
            grouped_KMeans_dict[label] = grouped_KMeans_dict[label] + roi_list
            
    return batch_labelled_total_list, grouped_KMeans_dict           
    

In [8]:
# def get_labels(plotting_dict):
#     label3 = []
#     label2 = []
#     label1 = []
#     label0 = []
#     for label, trace in zip(plotting_dict['Batched_total_labels'], plotting_dict['Batched_total_traces']):
#         if label == 2:
#             label2.append(trace)
#         elif label == 1:
#             label1.append(trace)
#         elif label == 3:
#             label3.append(trace)
#         else:
#             label0.append(trace)

#     label0_prop = int(10*(len(label0)/len(labelled_total_list)))
#     label1_prop = int(10*(len(label1)/len(labelled_total_list)))
#     label2_prop = int(10*(len(label2)/len(labelled_total_list)))
#     label3_prop = int(10*(len(label3)/len(labelled_total_list)))
#     label_props = [label0_prop, label1_prop, label2_prop]
#     print("The labels are label 0, 1, 2, 3 in order ", label0_prop, label1_prop, label2_prop, label3_prop)
#     return label0, label1, label2, label_props

In [9]:
def label_dict(plotting_dict):
    label_dict = {}
    for label, trace in zip(plotting_dict['Batched_total_labels'], plotting_dict['Batched_total_traces']):
        if label not in label_dict.keys():
            label_dict[label] = []
            label_dict[label].append(trace)

        else:
            label_dict[label].append(trace)
    return label_dict


In [10]:
def demo_main():
    plotting_dict = {}
    batched_original_data_dict, trial_names = get_batched_data(roi_paths)
    batch_total_sorted_temporal_traces, batch_total_sorted_kept_rois = get_batched_total_sorted_rois(batched_original_data_dict, video)
    batch_labelled_total_list, grouped_KMeans_dict = kMeans_clustering(batch_total_sorted_temporal_traces, batch_total_sorted_kept_rois, n_clusters)
    
    plotting_dict['Batched_total_labels'] = batch_labelled_total_list
    plotting_dict['Batched_total_traces'] = batch_total_sorted_temporal_traces
    print("main fun " +str(len(plotting_dict['Batched_total_labels'])))
    label_dictionary = label_dict(plotting_dict)
    return label_dictionary, batch_labelled_total_list, plotting_dict, grouped_KMeans_dict
# demo_main()

In [11]:
label_dictionary, batch_labelled_total_list, plotting_dict, grouped_KMeans_dict = demo_main()

Enter trials sequentially by letter: dc
main fun 354
Label not in keys, count is: 1
Label not in keys, count is: 2
Label not in keys, count is: 3
Label not in keys, count is: 4


C:\Users\mmart\anaconda3\envs\caiman\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\mmart\anaconda3\envs\caiman\lib\site-packages\numpy\lib\function_base.py:2400: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\mmart\anaconda3\envs\caiman\lib\site-packages\numpy\lib\function_base.py:2401: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\mmart\anaconda3\envs\caiman\lib\site-packages\sklearn\cluster\_kmeans.py:322: RuntimeWarning: invalid value encountered in true_divide
  max_iter=max_iter, verbose=verbose)
C:\Users\mmart\anaconda3\envs\caiman\lib\site-packages\sklearn\cluster\_kmeans.py:322: RuntimeWarning: invalid value encountered in true_divide
  max_iter=max_iter, verbose=verbose)
C:\Users\mmart\anaconda3\envs\caiman\lib\site-packages\sklearn\cluster\_kmeans.py:322: RuntimeWarning: invalid value encountered in true_divide
  max_iter=max_ite

In [13]:
# print(len(label_dictionary[0]))
# print(plotting_dict["Batched_total_traces"][0].shape)
# print(len(plotting_dict["Batched_total_traces"]))
# print(type(plotting_dict["Batched_total_labels"]))
# print(len(plotting_dict["Batched_total_labels"]))
# print(plotting_dict["Batched_total_labels"][0:10])
# print(len(label_dictionary[1]))
# print(len(label_dictionary[3]))
# print(len(label_dictionary[2]))
# print(len(label_dictionary[0]))
# print(type(plotting_dict['Batched_total_traces'][0][0]))
# print(plotting_dict['Batched_total_traces'][0][0])
# print(type(plotting_dict['Batched_total_labels']))
# print(len(plotting_dict))
# print(plotting_dict[])
# print(grouped_KMeans_dict.keys())
# print(batch_total_sorted_temporal_traces.keys())
# print(len(batched_original_data_dict['d'][0]))
# for i in range(len(grouped_KMeans_dict.keys())):
#     print(len(grouped_KMeans_dict[i]))
# print(plotting_dict["Batched_total_traces"][1])
# print(label_dictionary[2][1])
# print(plotting_dict["Batched_total_traces"][2])
# print(label_dictionary[2][2])

[2 2 2 2 3 3 3 3 3 2]
67
131
77
79
[-0.23726651 -0.28461465 -0.32909839 -0.3708912  -0.41015586 -0.44704513
 -0.48170289 -0.51426397 -0.54485534 -0.57359611 -0.60059827 -0.62596697
 -0.64980097 -0.67219317 -0.69323078 -0.7129957  -0.73156506 -0.74901097
 -0.76540161 -0.78080065 -0.79526816 -0.80886045 -0.82163052 -0.83362804
 -0.84489978 -0.85548969 -0.84202426 -0.85278811 -0.86290082 -0.87240173
 -0.88132793 -0.88971412 -0.897593   -0.90499524 -0.91194971 -0.50054874
 -0.44756421 -0.48219051 -0.51472213 -0.54528585 -0.57400054 -0.60097821
 -0.6263239  -0.65013636 -0.67250824 -0.69352679 -0.71327387 -0.24182696
 -0.28889924 -0.23121059 -0.27892504 -0.32375305  0.79451615  0.68475128
  0.58162665  0.48474027  0.39371504  0.30819635  0.22785094  0.152366
  0.88052182  1.28918947  1.14949978  3.40923597  3.14129597  3.00072372
  2.75749642  2.5289828   2.31429315  2.26877821  2.06982922  1.88291564
  1.70730907  1.54232584  1.38732283  1.24169682  1.10488006  0.97634028
  0.85557603  0.74

In [ ]:
def main():
#     plotting_dict = {}
#     data_retrieval                                    #done
#         get file names                                #done
#         get data from files                           #done
#     data_sorting                                      #done
#         sort data by z score per plane                #done
#         store sorted temporal data                    #done
#     elbow_plotting                                    #
#         elbow plot of data to be k-means              #
#     k_means_cluster                                   #done
#         k-means clustering on whatever scope of data  #done
#         store clusters/ labels                        #done
#     plot_activity                                     #
#         plot heatmap of clusters                      #
#     plot_anatomy                                      #
#         plot labeled image                            #
#     save figures                                      #
#     #use dict.update() to create the mega dictionary with everything nested appropriately
    plotting_dict = {}
    ##Do this one at a time, append sorted temp traces to a trial
        ##Then append sorted kept rois to same trial
    ##Create total z score and update plotting dict
    ##update plotting dict and add total labels
        ## append labels to trials
        
    batched_original_data_dict, trial_names = get_batched_data(roi_paths)
    batch_total_sorted_temporal_traces, batch_total_sorted_kept_rois = get_batched_total_sorted_rois(batched_original_data_dict, n_clusters, video)
    batch_labelled_total_list, grouped_KMeans_dict = kMeans_clustering(batch_total_sorted_temporal_traces, batch_total_sorted_kept_rois, n_clusters)
    
    
    plotting_dict['Batched_total_labels'] = batch_labelled_total_list
    plotting_dict['Batched_total_traces'] = batch_total_sorted_temporal_traces
    print(len(batch_labelled_total_list))
    print(type(batched_original_data_dict))
    print(type(trial_names))
    for i in trial_names:
        print(len(batched_original_data_dict[i][1][4]))#['removed_rois'])